## testes

In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

rdd = sc.parallelize([('X01',41,'US',3),
                       ('X01',41,'UK',1),
                       ('X01',41,'CA',2),
                       ('X02',72,'US',4),
                       ('X02',72,'UK',6),
                       ('X02',72,'CA',7),
                       ('X02',72,'XX',8)])

# convert to a Spark DataFrame                    
schema = StructType([StructField('ID', StringType(), True),
                     StructField('Age', IntegerType(), True),
                     StructField('Country', StringType(), True),
                     StructField('Score', IntegerType(), True)])

df = sqlContext.createDataFrame(rdd, schema)

In [2]:
df.show()

+---+---+-------+-----+
| ID|Age|Country|Score|
+---+---+-------+-----+
|X01| 41|     US|    3|
|X01| 41|     UK|    1|
|X01| 41|     CA|    2|
|X02| 72|     US|    4|
|X02| 72|     UK|    6|
|X02| 72|     CA|    7|
|X02| 72|     XX|    8|
+---+---+-------+-----+



In [3]:
data=[('X01',41,'US',3),('X01',41,'UK',1),('X01',41,'CA',2),('X02',72,'US',4),
      ('X02',72,'UK',6),('X02',72,'CA',7),('X02',72,'XX',8)]
calls = sc.parallelize(data,1)
countries = ['CA', 'UK', 'US', 'XX']

In [4]:
def reshape(t):
    out = []
    out.append(t[0])
    out.append(t[1])
    for v in brc.value:
        if t[2] == v:
            out.append(t[3])
        else:
            out.append(0)
    return (out[0],out[1]),(out[2],out[3],out[4],out[5])
def cntryFilter(t):
    if t[2] in brc.value:
        return t
    else:
        pass

def addtup(t1,t2):
    j=()
    for k,v in enumerate(t1):
        j=j+(t1[k]+t2[k],)
    return j

def seq(tIntrm,tNext):
    return addtup(tIntrm,tNext)

def comb(tP,tF):
    return addtup(tP,tF)


countries = ['CA', 'UK', 'US', 'XX']
brc = sc.broadcast(countries)
reshaped = calls.filter(cntryFilter).map(reshape)
pivot = reshaped.aggregateByKey((0,0,0,0),seq,comb,1)
for i in pivot.collect():
    print i

(('X02', 72), (7, 6, 4, 8))
(('X01', 41), (2, 1, 3, 0))


In [5]:
pivot.toDF().show()

+--------+---------+
|      _1|       _2|
+--------+---------+
|[X02,72]|[7,6,4,8]|
|[X01,41]|[2,1,3,0]|
+--------+---------+



In [6]:
# OU
from pyspark.sql.types import Row
callRow = calls.map(lambda t:   

Row(userid=t[0],age=int(t[1]),country=t[2],nbrCalls=t[3]))
callsDF = sqlContext.createDataFrame(callRow)
callsDF.printSchema()
callsDF.registerTempTable("calls")
res = sqlContext.sql("select userid,age,max(ca) as CA,max(uk) as UK,max(us) as US,max(xx) as XX\
                    from (select userid,age,\
                                  case when country='CA' then nbrCalls else 0 end ca,\
                                  case when country='UK' then nbrCalls else 0 end uk,\
                                  case when country='US' then nbrCalls else 0 end us,\
                                  case when country='XX' then nbrCalls else 0 end xx \
                             from calls) x \
                     group by userid,age")

root
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- nbrCalls: long (nullable = true)
 |-- userid: string (nullable = true)



In [7]:
res.show()

+------+---+---+---+---+---+
|userid|age| CA| UK| US| XX|
+------+---+---+---+---+---+
|   X01| 41|  2|  1|  3|  0|
|   X02| 72|  7|  6|  4|  8|
+------+---+---+---+---+---+



## dados de fraude

In [8]:
# nao considerar abaixo, só acima

In [9]:
#from pyspark.sql.types import Row
#allRow = calls.map(lambda t:   

#Row(agencia=t[0],atm=int(t[1]),evento=t[2],nbrCalls=t[3]))
#callsDF = sqlContext.createDataFrame(callRow)
#callsDF.printSchema()
#callsDF.toPandas()

In [10]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

rdd = sc.parallelize([('X01','41','1000',3.0),
                       ('X01','42','2000',1.0),
                       ('X01','43','3000',2.0),
                       ('X02','72','4000',4.0),
                       ('X02','73','5000',6.0),
                       ('X02','74','6000',7.0),
                       ('X02','75','7000',8.0)])

# convert to a Spark DataFrame                    
schema = StructType([StructField('agencia', StringType(), True),
                     StructField('atm', StringType(), True),
                     StructField('evento', StringType(), True),
                     StructField('iproxmed', DoubleType(), True)])

df = sqlContext.createDataFrame(rdd, schema)
df.toPandas()

agencia atm evento  iproxmed
0     X01  41   1000       3.0
1     X01  42   2000       1.0
2     X01  43   3000       2.0
3     X02  72   4000       4.0
4     X02  73   5000       6.0
5     X02  74   6000       7.0
6     X02  75   7000       8.0

In [11]:
df.registerTempTable("df")
res = sqlContext.sql("select agencia,atm,max(ev1000) as ev1000,max(ev2000) as ev2000,\
                    max(ev3000) as ev3000,max(ev4000) as ev4000\
                    from (select agencia,atm,\
                                  case when evento='1000' then iproxmed else 0 end ev1000,\
                                  case when evento='2000' then iproxmed else 0 end ev2000,\
                                  case when evento='3000' then iproxmed else 0 end ev3000,\
                                  case when evento='4000' then iproxmed else 0 end ev4000 \
                             from df) x \
                     group by agencia,atm")

In [12]:
res.toPandas()

agencia atm  ev1000  ev2000  ev3000  ev4000
0     X01  41     3.0     0.0     0.0     0.0
1     X01  42     0.0     1.0     0.0     0.0
2     X01  43     0.0     0.0     2.0     0.0
3     X02  72     0.0     0.0     0.0     4.0
4     X02  73     0.0     0.0     0.0     0.0
5     X02  74     0.0     0.0     0.0     0.0
6     X02  75     0.0     0.0     0.0     0.0

## mllib

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer


In [29]:
# Prepare training documents from a list of (id, text, label) tuples.

from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

rdd = sc.parallelize([(0L, "a b c d e spark", 1.0),
                       (1L, "b d", 0.0),
                       (2L, "spark f g h", 1.0),
                       (3L, "hadoop mapreduce", 0.0)])

# convert to a Spark DataFrame                    
schema = StructType([StructField("id", LongType(), True),
                     StructField('text', StringType(), True),
                     StructField('label', DoubleType(), True)])

training = sqlContext.createDataFrame(rdd, schema)

training.show()

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



In [39]:
# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenizer.getOutputCol()

'words'

In [30]:
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [31]:
# Fit the pipeline to training documents.
model = pipeline.fit(training)

In [38]:
# Prepare test documents, which are unlabeled (id, text) tuples.
#test = spark.createDataFrame([
#       (4L, "spark i j k"),
#        (5L, "l m n"),
#        (6L, "spark hadoop spark"),
#        (7L, "apache hadoop")], ["id", "text"])

rdd = sc.parallelize([(4L, "spark i j k"),
                       (5L, "l m n"),
                       (6L, "spark hadoop spark"),
                       (7L, "apache hadoop")])

# convert to a Spark DataFrame                    
schema = StructType([StructField("id", LongType(), True),
                     StructField('text', StringType(), True)])

test = sqlContext.createDataFrame(rdd, schema)
test.show()

+---+------------------+
| id|              text|
+---+------------------+
|  4|       spark i j k|
|  5|             l m n|
|  6|spark hadoop spark|
|  7|     apache hadoop|
+---+------------------+



In [33]:
# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.159640773879,0.840359226121], prediction=1.000000
(5, l m n) --> prob=[0.837832568548,0.162167431452], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.0692663313298,0.93073366867], prediction=1.000000
(7, apache hadoop) --> prob=[0.982157533344,0.0178424666556], prediction=0.000000


In [37]:
selected.show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.15964077387874...|       1.0|
|  5|             l m n|[0.83783256854766...|       0.0|
|  6|spark hadoop spark|[0.06926633132976...|       1.0|
|  7|     apache hadoop|[0.98215753334442...|       0.0|
+---+------------------+--------------------+----------+

